In [1]:
# %matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.font_manager as fm
[f.name for f in fm.fontManager.ttflist if "Garamond" in f.name]
from matplotlib import rcParams

sns.set_style('ticks', {"axes.linewidth": "1",'axes.yaxis.grid':False})

In [2]:
# from phys.plot_scripts.decoding_summary import PlotHelper
from utils.decoding_summary import PlotHelper

In [3]:
import sys
sys.path.append('./utils')

In [4]:
from utils import generic_plot_utils as rr_gpu

In [5]:
    def plot_decode_performance_unrolled_over_time(self, time_idx=None, compute_egocentric=False):

        if compute_egocentric:  ###
            beh_idx_mat = [6, 7]
            file_suffix = 'egocentric'
        else:
            beh_idx_mat = [0, 1]
            file_suffix = 'allocentric'
        def plot_average_metric(res):
            cmap = plt.get_cmap('Dark2', 8) # green, red, blue, magenta, brightGreen, yellow, 6brown, 7grey
            f, axes = plt.subplots(1, 3, figsize=(7.5, 3))
            for ax_i, met in enumerate(['r_mu', 'mae_mu', 'mse_mu']):
                ax = axes[ax_i]
                for beh_idx in beh_idx_mat: ####
                    mu, sig = [], []
                    x_for_stat = []
                    for mi, mask_fn in enumerate(['start_occ_pad0', 'occ_end_pad0']):
                        x = res[mask_fn][met][:, beh_idx]
                        mu.append(np.nanmean(x, axis=0))
                        sig.append(np.nanstd(x, axis=0))
                        x_for_stat.append(x)

                    # ranksums
                    # stat_object = ranksums(x_for_stat[0],x_for_stat[1])
                    stat_object = wilcoxon(x_for_stat[1])
                    p_value = stat_object.pvalue

                    # # permutation_test
                    # p_value = permutation_test(x_for_stat[0],x_for_stat[1],
                    #                            method='approximate',
                    #                            num_rounds=1000,
                    #                            seed=0)

                    print(mu)
                    print(sig)
                    ax.errorbar(range(2), mu, yerr=sig, fmt='-o', mfc='w', mec=cmap(beh_idx), color=cmap(beh_idx),label=f'p={p_value:.2E}') # ,label='$\it{p=%2.6f}$' % p_value)
                    ax.legend()
                    ax.set_xticks([0, 1])
                    ax.set_xticklabels(['Visible \n Epoch', 'Occluded \n Epoch'])
                    ax.set_xlim([-0.25, 1.25])
                    if met == 'r_mu':
                        ax.set_ylim([0, 1])
                    else:
                        ax.set_ylim([0, 10])
                    rr_gpu.make_axis_nice(ax)

            plt.tight_layout()
            return f, axes

        def plot_timecourse_of_error(res,subject_id):
            f, axes = plt.subplots(1, 4, figsize=(8, 4))
            ylim_all = [0, 7]
            for mfi, mask_fn in enumerate(['start_occ_pad0', 'occ_end_pad0']):
                rmse_alignstart = (res[mask_fn]['mse_t'][:, :, beh_idx_mat[0]] + res[mask_fn]['mse_t'][:, :, beh_idx_mat[1]]) ** 0.5 ####
                rmse_alignend = (res[mask_fn]['mse_t2'][:, :, beh_idx_mat[0]] + res[mask_fn]['mse_t2'][:, :, beh_idx_mat[1]]) ** 0.5

                if 'start' in mask_fn:
                    min_timesteps = 15
                else:
                    min_timesteps = 14

                ax = axes[mfi * 2]
                t = np.arange(0, min_timesteps, 1)
                ax.errorbar(t, np.nanmean(rmse_alignstart[:, t], axis=0),
                            np.nanstd(rmse_alignstart[:, t], axis=0), fmt='-ko')
                ax.set_ylim(ylim_all)
                ax.axvline(0, linestyle='--', color=[0.5, 0.5, 0.5])
                rr_gpu.make_axis_nice(ax)

                np.savetxt('%s_%s_alignstart_t.csv'%(subject_id,mask_fn), t, fmt='%5.3f', delimiter=',', newline='\n')
                np.savetxt('%s_%s_alignstart_mean.csv'%(subject_id,mask_fn), np.nanmean(rmse_alignstart[:, t], axis=0), fmt='%5.3f', delimiter=',', newline='\n')
                np.savetxt('%s_%s_alignstart_std.csv'%(subject_id,mask_fn), np.nanstd(rmse_alignstart[:, t], axis=0), fmt='%5.3f', delimiter=',', newline='\n')

                ax = axes[mfi * 2 + 1]
                t = np.arange(-min_timesteps, 0, 1)
                ax.errorbar(t + 1, np.nanmean(rmse_alignend[:, t], axis=0),
                            np.nanstd(rmse_alignend[:, t], axis=0), fmt='-ko')
                ax.set_ylim(ylim_all)
                ax.axvline(0, linestyle='--', color=[0.5, 0.5, 0.5])
                rr_gpu.make_axis_nice(ax)

                np.savetxt('%s_%s_alignend_t.csv'%(subject_id,mask_fn), t + 1, fmt='%5.3f', delimiter=',', newline='\n')
                np.savetxt('%s_%s_alignend_mean.csv'%(subject_id,mask_fn), np.nanmean(rmse_alignend[:, t], axis=0), fmt='%5.3f', delimiter=',', newline='\n')
                np.savetxt('%s_%s_alignend_std.csv'%(subject_id,mask_fn), np.nanstd(rmse_alignend[:, t], axis=0), fmt='%5.3f', delimiter=',', newline='\n')

            for i in [1, 2, 3]:
                axes[i].axes.get_yaxis().set_visible(False)

            return f, axes

        res_ = self.get_metrics_unrolled_and_remasked(time_idx=time_idx)
        print(res_.shape)

        f, axes = plot_average_metric(res_)
        outfn = '%s/%s_decode_perf_over_epochs_%s.pdf' % (self.figoutpath, self.save_prefix, file_suffix)
        f.savefig(outfn)

        f, axes = plot_timecourse_of_error(res_,self.subject_id)
        outfn = '%s/%s_decode_perf_over_time_%s.pdf' % (self.figoutpath, self.save_prefix, file_suffix)
        f.savefig(outfn)
        return

In [6]:
   for mfi, mask_fn in enumerate(['start_occ_pad0', 'occ_end_pad0']):
                print( mask_fn )

start_occ_pad0
occ_end_pad0


In [7]:
ph = PlotHelper(timebinsize=50,
                subject_id='all_hand_dmfc',
                masks_to_test_suffix='start_end_pad0',
                condition='occ',
                neural_data_to_use='neural_responses_reliable_FactorAnalysis_50',
                ncomp=50)

ph.load_data()
ph.load_ground_truth_data()
ph.get_colormap_for_decoded_variables()

self = ph

self.plot_decode_performance_unrolled_over_time(compute_egocentric=False)

../data//decode_all_hand_dmfc_occ_start_end_pad0_50ms_0.50_neural_responses_reliable_FactorAnalysis_50.pkl


NameError: name 'plt' is not defined

In [ ]:
res_ = self.get_metrics_unrolled_and_remasked(time_idx=None)

In [ ]:
res=res_

In [ ]:
 beh_idx_mat = [0, 1]

In [ ]:
subject_id='all_hand_dmfc'

In [ ]:
distance_matrix= np.load('distance_matrix.npy')

In [ ]:
import phys_utils
import dataset_augment_utils
import data_utils

In [ ]:
data = data_utils.load_neural_dataset(timebinsize=50,subject_id='all_hand_dmfc',recompute_augment=True)

data_aug = dataset_augment_utils.augment_data_structure(data)

In [ ]:
mask_start_occ = data_aug['masks']['occ']['start_occ_pad0']
mask_occ_end = data_aug['masks']['occ']['occ_end_pad0']

In [ ]:
mask_start_occ[1,23]

In [ ]:
distance_matrix.shape

In [ ]:
mask_new_star_occ= ~np.isnan(mask_start_occ) 

In [ ]:
star_occ = np.where(mask_new_star_occ,distance_matrix,np.nan) #7/18/2025

In [ ]:
a=mask_start_occ*distance_matrix #7/18/2025

In [ ]:
filtered_star_occ =star_occ[:,~np.isnan(star_occ).all(axis=0)] #7/18/2025

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(100)*50, np.nanmean(distance_matrix, axis=0))

plt.xlabel('Timepoint')
plt.ylabel('Distance')

In [ ]:
#Plot the avg ball pos acroos all the conditions for each valid timestamp #7/18/2025
plt.figure(figsize=(8, 4))
t = np.arange(6,6+ 15, 1)
plt.errorbar(t, np.nanmean(filtered_star_occ[:, t], axis=0),
                            np.nanstd(filtered_star_occ[:, t], axis=0), fmt='-ko')

plt.xlabel('Timepoint')
plt.ylabel('RMSE/Distance')
plt.title('Start of Trial')
plt.ylim(0,8)
plt.xlim(6,15+6)
plt.legend()
plt.show()

In [ ]:
#Plot the avg ball pos acroos all the conditions for each valid timestamp #7/18/2025
plt.figure(figsize=(8,4))
t = np.arange(13,37, 1)
plt.errorbar(t, np.nanmean(filtered_star_occ[:, t], axis=0),
                            np.nanstd(filtered_star_occ[:, t], axis=0), fmt='-ko')

plt.xlabel('Timepoint')
plt.ylabel('RMSE/Distance')
plt.title('Start of Occulsion')
plt.ylim(0,8)
plt.xlim(13,37)
plt.legend()
plt.show()

In [ ]:
mask_new_occ_end= ~np.isnan(mask_occ_end)  #7/18/2025
occ_end= np.where(mask_new_occ_end,distance_matrix,np.nan)
filtered_occ_end =occ_end[:,~np.isnan(occ_end).all(axis=0)]

In [ ]:
filtered_occ_end.shape

In [ ]:
#Plot the avg ball pos acroos all the conditions for each valid timestamp #7/18/2025
plt.figure(figsize=(8,4))
t = np.arange(0,14, 1)
plt.errorbar(t, np.nanmean(filtered_occ_end[:, t], axis=0),
                            np.nanstd(filtered_occ_end[:, t], axis=0), fmt='-ko')

plt.xlabel('Timepoint')
plt.ylabel('RMSE/Distance')
plt.title('End of Occulsion')
plt.ylim(0,8)
plt.xlim(0,14)
plt.legend()
plt.show()

In [ ]:
#Plot the avg ball pos acroos all the conditions for each valid timestamp #7/18/2025
plt.figure(figsize=(8,4))
t = np.arange(44,58, 1)
plt.errorbar(t, np.nanmean(filtered_occ_end[:, t], axis=0),
                            np.nanstd(filtered_occ_end[:, t], axis=0), fmt='-ko')

plt.xlabel('Timepoint')
plt.ylabel('RMSE/Distance')
plt.title('End of trial')
plt.ylim(0,8)
plt.xlim(44,58)
plt.legend()
plt.show()

In [ ]:
import os
print([f for f in os.listdir() if 'mean.csv' in f])

In [ ]:
time_idx = None
subject_id = 'all_hand_dmfc'

In [ ]:
#load and save code decoding_mean 
time_vectors_map = {
    'start_occ_pad0': {
        'start': np.arange(6, 6 + 15,1),
        'end': np.arange(13, 37,1)
    },
    'occ_end_pad0': {
        'start': np.arange(0, 14,1),
        'end': np.arange(44, 58,1)
    }
}


for mfi, mask_fn in enumerate(['start_occ_pad0', 'occ_end_pad0']):
    rmse_alignstart = (res[mask_fn]['mse_t'][:, :, beh_idx_mat[0]] + res[mask_fn]['mse_t'][:, :, beh_idx_mat[1]]) ** 0.5 ####
    rmse_alignend = (res[mask_fn]['mse_t2'][:, :, beh_idx_mat[0]] + res[mask_fn]['mse_t2'][:, :, beh_idx_mat[1]]) ** 0.5
    
    t_start = time_vectors_map[mask_fn]['start']
    t_end = time_vectors_map[mask_fn]['end']
    
    decoding_mean_start = np.nanmean(rmse_alignstart[:, t_start], axis=0)
    decoding_mean_end = np.nanmean(rmse_alignend[:, t_end], axis=0)
    
    np.save(f"{mask_fn}_start_decoding_mean.npy", decoding_mean_start)
    np.save(f"{mask_fn}_end_decoding_mean.npy", decoding_mean_end)

In [ ]:
font_path = 'EB_Garamond/EBGaramond-VariableFont_wght.ttf'
eb_garamond = fm.FontProperties(fname=font_path)

f, axes = plt.subplots(1, 4, figsize=(10.26, 9.62))
ylim_all = [0, 7]
subplot_titles = [
    'start-> +15 samples',
    '-14 samples-> occlusion onset',
    'occlusion onset-> +14',
    '-14 samples-> trial end'
]

for ax, title in zip(axes.flat, subplot_titles):
    ax.set_title(title, fontsize=40, pad=12)

for mfi, mask_fn in enumerate(['start_occ_pad0', 'occ_end_pad0']):
    rmse_alignstart = (res[mask_fn]['mse_t'][:, :, beh_idx_mat[0]] +
                       res[mask_fn]['mse_t'][:, :, beh_idx_mat[1]]) ** 0.5
    rmse_alignend = (res[mask_fn]['mse_t2'][:, :, beh_idx_mat[0]] +
                     res[mask_fn]['mse_t2'][:, :, beh_idx_mat[1]]) ** 0.5

    min_timesteps = 15 if 'start' in mask_fn else 14

    # === START-ALIGNED ===
    ax = axes[mfi * 2]
    t_start = np.arange(0, min_timesteps)

    ax.errorbar(t_start,
                np.nanmean(rmse_alignstart[:, t_start], axis=0),
                np.nanstd(rmse_alignstart[:, t_start], axis=0),
                fmt='-mo', label='Neural Decoding')

    if 'start' in mask_fn:
        baseline_start = np.nanmean(filtered_star_occ[:, t_start], axis=0)
    else:
        baseline_start = np.nanmean(filtered_occ_end[:, t_start], axis=0)

    ax.plot(t_start, baseline_start, '--o', color='orange', label='Baseline')

    ax.set_ylim(ylim_all)
    ax.set_xlim([0, 15])  # Make sure tick 15 shows
    ax.set_xticks([0, 5, 10, 15])
    ax.set_xticklabels(['0', '5', '10', '15'])
    ax.axvline(0, linestyle='--', color='gray')

    ax.set_title(subplot_titles[mfi * 2], fontproperties=eb_garamond, fontsize=17)
    ax.tick_params(axis='both', labelsize=20)

    # Format tick labels
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontproperties(eb_garamond)
        label.set_weight('bold')
        label.set_fontsize(15)

    if mfi == 0:
        ax.set_ylabel('RMSE', fontproperties=eb_garamond, fontsize=28)

    ax.legend(prop=eb_garamond, fontsize=16)

    # === END-ALIGNED ===
    ax = axes[mfi * 2 + 1]
    t_end = np.arange(0, min_timesteps)

    ax.errorbar(t_end - min_timesteps + 1,
                np.nanmean(rmse_alignend[:, -min_timesteps:], axis=0),
                np.nanstd(rmse_alignend[:, -min_timesteps:], axis=0),
                fmt='-mo', label='Neural Decoding')

    if 'start' in mask_fn:
        baseline_end = np.nanmean(filtered_star_occ[:, -min_timesteps:], axis=0)
    else:
        baseline_end = np.nanmean(filtered_occ_end[:, -min_timesteps:], axis=0)

    ax.plot(t_end - min_timesteps + 1, baseline_end, '--o', color='orange', label='Baseline')

    ax.set_ylim(ylim_all)
    ax.set_xlim([-14, 0])
    ax.set_xticks([-15, -10, -5, 0])
    ax.set_xticklabels(['-15', '-10', '-5', '0'])
    ax.axvline(0, linestyle='--', color='gray')

    ax.set_title(subplot_titles[mfi * 2 + 1], fontproperties=eb_garamond, fontsize=17)
    ax.set_xlabel('Time (samples)', fontproperties=eb_garamond, fontsize=28)
    ax.tick_params(axis='both', labelsize=20)

    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontproperties(eb_garamond)
        label.set_weight('bold')
        label.set_fontsize(15)

    if mfi == 0:
        ax.set_ylabel('RMSE', fontproperties=eb_garamond, fontsize=28)

    ax.legend(prop=eb_garamond, fontsize=16)

# Hide y-axis on right 3 plots
for i in [1, 2, 3]:
    axes[i].get_yaxis().set_visible(False)

plt.tight_layout()
plt.savefig("decoded_vs_basline.png", dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
data = np.load('/home/jcleckle/DMFC-decoder-simulation/code/distance_matrix.npy', allow_pickle=True)
print(data)

In [ ]:
ax.errorbar(t, np.nanmean(filtered_star_occ[:, t], axis=0),
                np.nanstd(filtered_star_occ[:, t], axis=0), fmt='-mo')
    ax.set_ylim(ylim_all)
    ax.axvline(0, linestyle='--', color=[0.5, 0.5, 0.5])
    rr_gpu.make_axis_nice(ax)
    
            
    ax = axes[mfi * 2 + 1]
    t = np.arange(-min_timesteps, 0, 1)
    ax.errorbar(t, np.nanmean(filtered_occ_end[:, t], axis=0),
                np.nanstd(filtered_occ_end[:, t], axis=0), fmt='-mo')
    ax.set_ylim(ylim_all)
    ax.axvline(0, linestyle='--', color=[0.5, 0.5, 0.5])
    rr_gpu.make_axis_nice(ax)

for i in [1, 2, 3]:
    axes[i].axes.get_yaxis().set_visible(False)

plt.tight_layout()
plt.show()


res = self.get_metrics_unrolled_and_remasked(time_idx=time_idx)


#f, axes = plot_average_metric(res)

#f, axes = plot_timecourse_of_error(res_,self.subject_i

In [ ]:

res_remasked = self.get_predictions_unrolled_and_remasked_base(time_idx=time_idx)

# # get x/y for true and predicted, shape = (23, 79, 100)
# true_all = res_remasked['true']['start_end_pad0']
# pred_all = res_remasked['pred']['start_end_pad0']

# mean over trials, result: (79, 100)
true_x = np.nanmean(res_remasked['true']['start_end_pad0'][:, 0, :, :], axis=0)
true_y = np.nanmean(res_remasked['true']['start_end_pad0'][:, 1, :, :], axis=0)

neural_prediction_x = np.nanmean(res_remasked['pred']['start_end_pad0'][:, 0, :, :], axis=0)
neural_prediction_y = np.nanmean(res_remasked['pred']['start_end_pad0'][:, 1, :, :], axis=0)


task_indices = [0, 54, 78]

fig, axs = plt.subplots(len(task_indices), 1, figsize=(13.92, 9.96*len(task_indices)), sharex=True, sharey=True)

for i, task_idx in enumerate(task_indices):
    ax = axs[i]
    ax.plot(true_x[task_idx], true_y[task_idx], 'k-', label='True trajectory')
    ax.plot(neural_prediction_x[task_idx], neural_prediction_y[task_idx], 'r--', label='Neural prediction')
    ax.set_title(f'Task condition {task_idx}')
    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    ax.legend()
    ax.grid(True)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()


In [ ]:
font_path = 'EB_Garamond/EBGaramond-VariableFont_wght.ttf'
eb_garamond = fm.FontProperties(fname=font_path)

rcParams['font.family'] = 'EB Garamond'

res_remasked = self.get_predictions_unrolled_and_remasked_base(time_idx=time_idx)

# Extract true and predicted mean trajectories
true_x = np.nanmean(res_remasked['true']['start_end_pad0'][:, 0, :, :], axis=0)
true_y = np.nanmean(res_remasked['true']['start_end_pad0'][:, 1, :, :], axis=0)
neural_prediction_x = np.nanmean(res_remasked['pred']['start_end_pad0'][:, 0, :, :], axis=0)
neural_prediction_y = np.nanmean(res_remasked['pred']['start_end_pad0'][:, 1, :, :], axis=0)

# Define task indices and their visual colors
task_indices = [0, 23, 54, 78]
colors = ['orange', 'purple', 'yellow', 'magenta']

fig, ax = plt.subplots(figsize=(9.55, 11.6))


for task_idx, color in zip(task_indices, colors):
    # 1. True trajectory → black dashed outline behind color solid line
    ax.plot(true_x[task_idx], true_y[task_idx],
            linestyle='--', color='black', linewidth=4, zorder=1)
    ax.plot(true_x[task_idx], true_y[task_idx],
            linestyle='-', color=color, linewidth=2, zorder=2)

    # 2. Neural prediction → dashed line in same color
    ax.plot(neural_prediction_x[task_idx], neural_prediction_y[task_idx],
            linestyle='--', color=color, linewidth=2, zorder=0)

# ax.set_title("True vs Neural Predicted Trajectories")
ax.set_xlabel("X Position", fontproperties=eb_garamond, fontsize=30)
ax.set_ylabel("Y Position", fontproperties=eb_garamond, fontsize=30)

ax.tick_params(axis='both', which='major', labelsize=23)
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontproperties(eb_garamond)
    label.set_fontweight('bold')
    label.setfontsize(15)


ax.legend(prop=eb_garamond)
ax.grid(True)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()



In [ ]:
# Number of timepoints and task conditions
n_conditions = 79
n_timepoints = 100

# Create synthetic "true" trajectory: flat line y=3 for all conditions and timepoints
true_y = np.full((n_conditions, n_timepoints), 3)

# Create synthetic "predicted" trajectory: flat line y=1 for all conditions and timepoints
predicted_y = np.full((n_conditions, n_timepoints), 1)

# Compute RMSE between true and predicted (only y here)
rmse = np.sqrt(np.nanmean((true_y - predicted_y)**2))

print("RMSE for synthetic flat lines:", rmse)
# Expected output: RMSE = 2.0

In [ ]:
print("res_remasked['true']['start_end_pad0'].shape:")
print(res_remasked['true']['start_end_pad0'].shape)

print("res_remasked['pred']['start_end_pad0'].shape:")
print(res_remasked['pred']['start_end_pad0'].shape)

In [ ]:
for mask_name in self.ground_truth['masks']:
    mask = np.array(self.ground_truth['masks'][mask_name])
    print(f"Mask '{mask_name}': sum = {np.sum(mask)}, shape = {mask.shape}")

In [ ]:
filtered_star_occ.shape

In [ ]:
filtered_occ_end.shape

In [ ]:
t.shape

In [ ]:
print(rmse_alignstart.shape)
print(rmse_alignend.shape)